# Data Collection Notebook

## Objectives
- Fetch data from Kaggle
- Save it as raw data
- Inspect the data
- Save it under outputs/datasets/collection

## Inputs
- Kaggle JSON file - the authentication token

## Outputs 
- Generate dataset: outputs/datasets/collection/HousingMarket.csv

---

## Change working directory
Changing current working directory to its parent folder

In [ ]:
import os 
cwd = os.getcwd()
cwd

In [ ]:
os.chdir(os.path.dirname(cwd))
print("You set a new current working directory")

In [ ]:
cwd = os.getcwd()
cwd

## Fetch data from Kaggle
Install Kaggle package to fetch data

In [ ]:
%pip install kaggle==1.5.12

Recognise token

In [ ]:
import os 
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

Define Kaggle dataset and destination folder

In [ ]:
KaggleDatasetPath = "codeinstitute/housing-prices-data"
DestinationFolder = "inputs/datasets/raw"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

Unzip downloaded file, delete the zip file, delete the kaggle.json file

In [ ]:
! unzip {DestinationFolder}/*.zip -d {DestinationFolder} \
  && rm {DestinationFolder}/*.zip \
  && rm kaggle.json

In [ ]:
---

## Load and Inspect Kaggle data

In [15]:
import pandas as pd
df_house_prices = pd.read_csv(f"inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/house_prices_records.csv")
df_house_prices.head()

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,...,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
0,856,854.0,3.0,No,706,GLQ,150,0.0,548,RFn,...,65.0,196.0,61,5,7,856,0.0,2003,2003,208500
1,1262,0.0,3.0,Gd,978,ALQ,284,NaN,460,RFn,...,80.0,0.0,0,8,6,1262,NaN,1976,1976,181500
2,920,866.0,3.0,Mn,486,GLQ,434,0.0,608,RFn,...,68.0,162.0,42,5,7,920,NaN,2001,2002,223500
3,961,NaN,NaN,No,216,ALQ,540,NaN,642,Unf,...,60.0,0.0,35,5,7,756,NaN,1915,1970,140000
4,1145,NaN,4.0,Av,655,GLQ,490,0.0,836,RFn,...,84.0,350.0,84,5,8,1145,NaN,2000,2000,250000


In [16]:
df_inherited = pd.read_csv(f"inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/inherited_houses.csv")
df_inherited.head()

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,...,LotArea,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd
0,896,0,2,No,468.0,Rec,270.0,0,730.0,Unf,...,11622,80.0,0.0,0,6,5,882.0,140,1961,1961
1,1329,0,3,No,923.0,ALQ,406.0,0,312.0,Unf,...,14267,81.0,108.0,36,6,6,1329.0,393,1958,1958
2,928,701,3,No,791.0,GLQ,137.0,0,482.0,Fin,...,13830,74.0,0.0,34,5,5,928.0,212,1997,1998
3,926,678,3,No,602.0,GLQ,324.0,0,470.0,Fin,...,9978,78.0,20.0,36,6,6,926.0,360,1998,1998


In [17]:
df_house_prices.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   1stFlrSF       1460 non-null   int64  
 1   2ndFlrSF       1374 non-null   float64
 2   BedroomAbvGr   1361 non-null   float64
 3   BsmtExposure   1422 non-null   object 
 4   BsmtFinSF1     1460 non-null   int64  
 5   BsmtFinType1   1315 non-null   object 
 6   BsmtUnfSF      1460 non-null   int64  
 7   EnclosedPorch  136 non-null    float64
 8   GarageArea     1460 non-null   int64  
 9   GarageFinish   1225 non-null   object 
 10  GarageYrBlt    1379 non-null   float64
 11  GrLivArea      1460 non-null   int64  
 12  KitchenQual    1460 non-null   object 
 13  LotArea        1460 non-null   int64  
 14  LotFrontage    1201 non-null   float64
 15  MasVnrArea     1452 non-null   float64
 16  OpenPorchSF    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  OverallQ

In [18]:
df_inherited.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   1stFlrSF       4 non-null      int64  
 1   2ndFlrSF       4 non-null      int64  
 2   BedroomAbvGr   4 non-null      int64  
 3   BsmtExposure   4 non-null      object 
 4   BsmtFinSF1     4 non-null      float64
 5   BsmtFinType1   4 non-null      object 
 6   BsmtUnfSF      4 non-null      float64
 7   EnclosedPorch  4 non-null      int64  
 8   GarageArea     4 non-null      float64
 9   GarageFinish   4 non-null      object 
 10  GarageYrBlt    4 non-null      float64
 11  GrLivArea      4 non-null      int64  
 12  KitchenQual    4 non-null      object 
 13  LotArea        4 non-null      int64  
 14  LotFrontage    4 non-null      float64
 15  MasVnrArea     4 non-null      float64
 16  OpenPorchSF    4 non-null      int64  
 17  OverallCond    4 non-null      int64  
 18  OverallQual   

Initial Observations:
- 1460 entries and 24 columns
- Target variable = SalePrice
- Mostly numerical columns, with a few categorical columns
- Missing data for the following features: 2ndFlrSF, BedroomAbvGr, BsmtExposure, BsmtFinType1, EnclosedPorch, GarageFinish, GarageYrBlt, LotFrontage, MasVnrArea, WoodDeckSF
- Missing data - severe for EnclosedPorch and WoodDeckSF
- GarageYrBlt, YearBuilt and YearRemodAdd are stored as integers - could derive age values from these columns
- Data on inherited houses is complete 



Deriving Useful Variables:
- HouseAge - age of the house in years 
- RemodAge - years since last remodel
- TotalSF - total internal square footage
- AboveGradeSF - total internal above grade square footage

In [19]:
from datetime import datetime

current_year = datetime.now().year

# df_house_prices
df_house_prices['HouseAge'] = current_year - df_house_prices['YearBuilt']
df_house_prices['RemodAge'] = current_year - df_house_prices['YearRemodAdd']
df_house_prices['GarageAge'] = current_year - df_house_prices['GarageYrBlt']
df_house_prices['TotalSF'] = df_house_prices['TotalBsmtSF'] + df_house_prices['1stFlrSF'] + df_house_prices['2ndFlrSF'].fillna(0)
df_house_prices['AboveGradeSF'] = df_house_prices['1stFlrSF'] + df_house_prices['2ndFlrSF'].fillna(0)


# df_inherited
df_inherited['HouseAge'] = current_year - df_inherited['YearBuilt']
df_inherited['RemodAge'] = current_year - df_inherited['YearRemodAdd']
df_inherited['GarageAge'] = current_year - df_inherited['GarageYrBlt']
df_inherited['TotalSF'] = df_inherited['TotalBsmtSF'] + df_inherited['1stFlrSF'] + df_inherited['2ndFlrSF'].fillna(0)
df_inherited['AboveGradeSF'] = df_inherited['1stFlrSF'] + df_inherited['2ndFlrSF'].fillna(0)

Creating Flags:
- IsRemodeled - boolean variable to distinguish remodeled vs original condition
- Has2ndFlr: 1=has second floor, 0=does not have second floor
- HasPorch: 1=has enclosed porch, 0=does not have enclosed porch
- HasDeck: 1=has wood deck, 0=does not have wood deck

In [20]:
# df_house_prices
df_house_prices['IsRemodeled'] = (df_house_prices['YearBuilt'] != df_house_prices['YearRemodAdd']).astype(int)
df_house_prices['Has2ndFlr'] = (df_house_prices['2ndFlrSF'] > 0).astype(int)
df_house_prices['HasPorch'] = (df_house_prices['EnclosedPorch'].fillna(0) > 0).astype(int)
df_house_prices['HasDeck'] = (df_house_prices['WoodDeckSF'].fillna(0) > 0).astype(int)

# df_inherited
df_inherited['IsRemodeled'] = (df_inherited['YearBuilt'] != df_inherited['YearRemodAdd']).astype(int)
df_inherited['Has2ndFlr'] = (df_inherited['2ndFlrSF'] > 0).astype(int)
df_inherited['HasPorch'] = (df_inherited['EnclosedPorch'].fillna(0) > 0).astype(int)
df_inherited['HasDeck'] = (df_inherited['WoodDeckSF'].fillna(0) > 0).astype(int)

Dropping year columns - unhelpful to train a model

In [ ]:
df_house_prices.drop(['YearBuilt', 'YearRemodAdd', 'GarageYrBlt'], axis=1, inplace=True)
df_inherited.drop(['YearBuilt', 'YearRemodAdd', 'GarageYrBlt'], axis=1, inplace=True)

df_house_prices.head()

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,...,SalePrice,HouseAge,RemodAge,GarageAge,TotalSF,AboveGradeSF,IsRemodeled,Has2ndFlr,HasPorch,HasDeck
0,856,854.0,3.0,No,706,GLQ,150,0.0,548,RFn,...,208500,22,22,22.0,2566.0,1710.0,0,1,0,0
1,1262,0.0,3.0,Gd,978,ALQ,284,NaN,460,RFn,...,181500,49,49,49.0,2524.0,1262.0,0,0,0,0
2,920,866.0,3.0,Mn,486,GLQ,434,0.0,608,RFn,...,223500,24,23,24.0,2706.0,1786.0,1,1,0,0
3,961,NaN,NaN,No,216,ALQ,540,NaN,642,Unf,...,140000,110,55,27.0,1717.0,961.0,1,0,0,0
4,1145,NaN,4.0,Av,655,GLQ,490,0.0,836,RFn,...,250000,25,25,25.0,2290.0,1145.0,0,0,0,0


## Create output file

In [ ]:
import pandas as pd

print(pd.__version__)

In [ ]:
%pip install pathLib

In [24]:
from pathlib import Path

out_dir = Path("outputs/datasets/collection")
out_dir.mkdir(parents=True, exist_ok=True)

df_house_prices.to_csv(out_dir / "HousingPrices.csv", index=False)
df_inherited.to_csv(out_dir / "InheritedHouses.csv", index=False)